In [32]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import time
import numpy as np

In [33]:
def get_all_string_after(string):
    split = string.split(':')
    res = split[1:len(split)]
    res = ''.join(res)
    return res,split[0]

def plain_text(string):
    return ''.join(char for char in string if char.isalnum())
def plain_text_arr(list_string,cut = False):
    res_list = []
    for string in list_string:
        string=plain_text(string)
        if(cut):
            res_list.append(cut_half(string))
        else:
            res_list.append(string)
    return res_list
     
def cut_half(string):
    half = int(len(string)/2)
    return string[0:half]

In [10]:
req = requests.get("https://myanimelist.net/topanime.php")
req

<Response [200]>

In [11]:
soup = bs(req.content,'html.parser')

In [12]:
list_items = soup.select("tr.ranking-list")  


In [48]:
user_score =[]
popularity=[]
members=[]
seasone_aired=[]
year=[]
brodcast_type=[]
studio=[]
name=[]

episodes=[]
source=[]
genre=[]
# standalone=[]

all_list = [name,user_score,popularity,members,seasone_aired,year,brodcast_type,studio,
           episodes,source,genre]


In [49]:
pages = 300
dataframe = pd.DataFrame({'name':[],
                          "score":[],
                          'popularity':[],
                          'members':[],
                          'season':[],
                          'year':[],
                          'type':[],
                          'studio':[],
                          'episode':[],
                          'source':[],
                          'genre':[]})

for i in range(0,pages) :
    time.sleep(1)
    req = requests.get("https://myanimelist.net/topanime.php?limit=" + str(i*50))
    print(i)
    soup = bs(req.content,'html.parser')
    list_items = soup.select("tr.ranking-list")
    
    for item in list_items:
        time.sleep(3)
        link = item.a["href"]
        print(link)
        req = requests.get(link)
        soup = bs(req.content,'html.parser')
        year_bool = False
        
        try:
            user_score.append(soup.select('div.score-label')[0].string)
        except:
            user_score.append(np.nan)
                
        try:
            name.append(soup.select('div.h1-title')[0].strong.string)
        except:
            name.append(None)
        
        try:
            popularity.append(plain_text(soup.select('span.popularity')[0].strong.string))
        except:
            popularity.append(np.nan)
        
        try:
            members.append(soup.select('span.members')[0].strong.get_text().replace(',',''))
        except:
            members.append(np.nan)
        try:
            seasone_aired.append(soup.select('span.season')[0].a.get_text())
        except:
            seasone_aired.append(None)
        try:
            year.append(soup.select('span.season')[0].a.get_text().split(" ")[1])
        except:
            year_bool=True
#             year.append(np.nan)
            
        try:
            brodcast_type.append(soup.select('span.type')[0].a.get_text())
        except:
            brodcast_type.append(None)
        
        try:
            studio.append(soup.select('span.studio')[0].a.get_text())
        except:
            studio.append(None)
        
        other_data_soup = soup.select('div.spaceit_pad')
        episode_bool = False
        source_bool = False
        genre_bool = False
        year2_bool = False
        for row in other_data_soup:
            field_text,field_name =get_all_string_after(row.get_text()) 
            field_name =plain_text(field_name)
            
            if(field_name=='Episodes'):
                field_text=plain_text(field_text)
                episodes.append(field_text)
                episode_bool=True
            elif(field_name=='Source'):
                field_text=plain_text(field_text)
                source.append(field_text)
                source_bool=True
            elif(field_name=='Genres'):
                field_text = plain_text_arr(field_text.split(','),cut=True)
                genre.append(field_text)
                genre_bool=True
            if(year_bool and field_name=="Aired"):
                year2_bool = True
                field_text = plain_text_arr(field_text.split(','))
                try:
                    year.append(field_text[1].split('t')[0])
                except:
                    year.append(np.nan)
                
                
            
        
                        
        if(episode_bool == False):
            episodes.append(None)
        if(source_bool == False):
            source.append(None)
        if(genre_bool == False):
            genre.append(None)
        if(year2_bool==False and year_bool):
            year.append(np.nan)
        
    
    for i in range(len(all_list)-1):
        print(len(all_list[i]))

    df = pd.concat([dataframe, pd.DataFrame({'name':name,
                                             "score":user_score,
                                             'popularity':popularity,
                                             'members':members,
                                             'season':seasone_aired,
                                             'year':year,
                                             'type':brodcast_type,
                                             'studio':studio,
                                             'episode':episodes,
                                             'source':source,
                                             'genre':genre})])
    df.to_csv('ver2_all_anime.csv')
    


218
https://myanimelist.net/anime/23375/Minarai_Diva
https://myanimelist.net/anime/22467/Minihams_no_Ai_no_Uta
https://myanimelist.net/anime/34519/Mobile_Suit_Gakuen__G-Reco_Koushien
https://myanimelist.net/anime/27437/Mushiba_Tetsudou
https://myanimelist.net/anime/14743/Nara_Shika_Monogatari
https://myanimelist.net/anime/19755/Oshiri_Kajiri_Mushi_TV_2nd_Season
https://myanimelist.net/anime/14023/R²__Rise_R_to_the_Second_Power
https://myanimelist.net/anime/32583/Sense_Honma_ni_Honma
https://myanimelist.net/anime/1285/Soukihei_MD_Geist
https://myanimelist.net/anime/38863/The_Laughing_Spider
https://myanimelist.net/anime/31476/Wake_up_Tamala
https://myanimelist.net/anime/7721/Yans_Gans_Meat_or_Die
https://myanimelist.net/anime/31170/Youkai_Ninpouchou_Jiraiya
https://myanimelist.net/anime/30169/Yubikiri
https://myanimelist.net/anime/46004/243_Mini_Anime__Inside_Story
https://myanimelist.net/anime/25971/Angel_Special
https://myanimelist.net/anime/32886/Anitore_EX__Jigoku_no_Training
https:

https://myanimelist.net/anime/43745/Attack_Zero_x_Evangelion
https://myanimelist.net/anime/38047/Crescendo_Story
https://myanimelist.net/anime/6939/Eien_no_Hou
https://myanimelist.net/anime/33114/Experiment
https://myanimelist.net/anime/34910/Heya_Music
https://myanimelist.net/anime/13085/Kitsutsuki_Keikaku
https://myanimelist.net/anime/14563/Maji_de_Otaku_na_English_Ribbon-chan__Eigo_de_Tatakau_Mahou_Shoujo
150
150
150
150
150
150
150
150
150
150
221
https://myanimelist.net/anime/10979/48x61
https://myanimelist.net/anime/13971/663114
https://myanimelist.net/anime/24861/Akaoni_to_Aooni_no_Tango
https://myanimelist.net/anime/28245/Ari-chan
https://myanimelist.net/anime/39214/Asa_wo_Kitan
https://myanimelist.net/anime/34992/Byoudoku_no_Denpa
https://myanimelist.net/anime/28057/Dangobei_Torimonochou__Hirake_-_Goma_no_Maki
https://myanimelist.net/anime/38795/I_Want_Nothing
https://myanimelist.net/anime/18375/Kaitou_Reinya_Pilot
https://myanimelist.net/anime/21717/Koisuru_Niwatori
https://m

https://myanimelist.net/anime/36595/Fortissimo_Bell
https://myanimelist.net/anime/23909/Hoshi_no_Yuuenchi
https://myanimelist.net/anime/33698/IRoid__Koi_no_Yuukou_Frontier
https://myanimelist.net/anime/5581/Kaitei_Daisensou__Ai_no_20000_Miles
https://myanimelist.net/anime/28047/Kokoro_no_Chikara
https://myanimelist.net/anime/30947/Kurayami_Santa
https://myanimelist.net/anime/39035/Lux_Straight___Beauty_Massugu_Ikiru
https://myanimelist.net/anime/34625/Master_of_Torque__Epilogue
https://myanimelist.net/anime/29767/Minamo
https://myanimelist.net/anime/2765/Momotarou__Umi_no_Shinpei
https://myanimelist.net/anime/14125/Otogi_no_Kuni_no_Birthday
https://myanimelist.net/anime/28687/Poker
https://myanimelist.net/anime/33904/Suntory_Tennensui_CMs
https://myanimelist.net/anime/40105/Superfluid
300
300
300
300
300
300
300
300
300
300
224
https://myanimelist.net/anime/10743/Tora-chan_no_Kankan_Mushi
https://myanimelist.net/anime/30336/Tori
https://myanimelist.net/anime/30075/Yuki_no_Hi_no_Tayori


https://myanimelist.net/anime/38449/Choujigen_Kakumei_Anime__Dimension_High_School
https://myanimelist.net/anime/40687/Cleaning_Switch
https://myanimelist.net/anime/36264/Drive_Car
https://myanimelist.net/anime/32775/Gene
https://myanimelist.net/anime/29665/Genei_Toshi
https://myanimelist.net/anime/5799/Kaede_New_Town
https://myanimelist.net/anime/2592/Kimera
https://myanimelist.net/anime/7025/Kiseichuu_no_Ichiya
https://myanimelist.net/anime/20725/Lies_Lies
https://myanimelist.net/anime/3026/Compiler
https://myanimelist.net/anime/38758/Cup_of_Smoke
https://myanimelist.net/anime/5838/Furudera_no_Obake-soudou
https://myanimelist.net/anime/37550/Gingirl
https://myanimelist.net/anime/39620/Hakata_Mentai_Pirikarako-chan
https://myanimelist.net/anime/31584/Henkei_Sakuhin_Dai_1-ban
https://myanimelist.net/anime/38632/Honda-san_x_Taka_no_Tsume
https://myanimelist.net/anime/29651/Ikedaya_Soudou
https://myanimelist.net/anime/35400/Kamishibai_Kintarou_no_Maki
450
450
450
450
450
450
450
450
450


https://myanimelist.net/anime/8733/Tousan_no_Tsukutta_Uta
https://myanimelist.net/anime/36764/Tripod_Baby
https://myanimelist.net/anime/29649/Tsuiseki
https://myanimelist.net/anime/31269/Venus
https://myanimelist.net/anime/33124/Wakuwaku♥Nukegake_Daisakusen
https://myanimelist.net/anime/36092/Alluptoyou
https://myanimelist.net/anime/37143/Alphard_x_Taka_no_Tsume
https://myanimelist.net/anime/28305/Broots
https://myanimelist.net/anime/1560/Choujin_Gakuen_Gowcaizer__The_Voltage_Fighters
https://myanimelist.net/anime/1705/Genma_Taisen__Shinwa_Zenya_no_Shou
https://myanimelist.net/anime/3516/Hal___Bons
https://myanimelist.net/anime/6090/Hon_Ran
https://myanimelist.net/anime/28067/Kyouryoku_Boukuusen
https://myanimelist.net/anime/23387/Master_of_Torque
https://myanimelist.net/anime/25265/Megumi_to_Taiyou__Kajuu_Gummi_Tweet_Love_Story
https://myanimelist.net/anime/32737/009_Re_Cyborg_x_Zip
https://myanimelist.net/anime/37729/Akuma_no_Organ
https://myanimelist.net/anime/29865/BAR_Kiraware_Yas

https://myanimelist.net/anime/10105/Bavi_Stock
https://myanimelist.net/anime/32765/Cherry_Blossom_Music
https://myanimelist.net/anime/24843/Clarinet_Kowashichatta
https://myanimelist.net/anime/49292/Deep_Insanity__The_Lost_Child
https://myanimelist.net/anime/36113/Love
https://myanimelist.net/anime/39096/Danbou_to_Reibou
https://myanimelist.net/anime/35999/Dia_Horizon_Kabu
https://myanimelist.net/anime/30664/Jinkou_no_Rakuen
https://myanimelist.net/anime/33401/Momotarou_Music
https://myanimelist.net/anime/3695/Spectral_Force_Chronicle_Divergence
https://myanimelist.net/anime/14529/Super_Majingga_3
https://myanimelist.net/anime/30090/Zombie_Ehon
https://myanimelist.net/anime/35224/Mobile_Suit_Gundam__Twilight_Axis
https://myanimelist.net/anime/29391/Pinky_Wakuwaku_Tabi_Stroll
https://myanimelist.net/anime/37571/Senjuushi
https://myanimelist.net/anime/31760/Tsuru_Shitae_Waka_Kan
https://myanimelist.net/anime/30559/Why
https://myanimelist.net/anime/35432/Yi_Wan_Nian_Yi_Hou
https://myanime

https://myanimelist.net/anime/32778/Ame_no_Bus_Stop-hen
https://myanimelist.net/anime/35388/Ameya_Tanuki
https://myanimelist.net/anime/5934/Greed
https://myanimelist.net/anime/1647/Kakugo_no_Susume
https://myanimelist.net/anime/31267/Akuma_no_Kairozu
https://myanimelist.net/anime/33245/Ame-iro_Cocoa_in_Hawaii
https://myanimelist.net/anime/8661/Chimuchimucheri_ChimChimCheree
https://myanimelist.net/anime/29768/Fantastic_Cell
https://myanimelist.net/anime/22417/Quantum_Leap
https://myanimelist.net/anime/10763/Robot_Taekwon_V
https://myanimelist.net/anime/34393/Room_Mate
https://myanimelist.net/anime/34721/Samuroid_Zero
https://myanimelist.net/anime/7900/Super_Child
https://myanimelist.net/anime/30400/Yakusoku
https://myanimelist.net/anime/38017/Yuukai_Anna
https://myanimelist.net/anime/8604/Oshiri_Kajiri_Mushi
https://myanimelist.net/anime/35923/Ame-iro_Cocoa_Series__Ame-con
https://myanimelist.net/anime/30093/Ball_yo_Doko_e_Yuku
https://myanimelist.net/anime/33541/Bishoujo_Yuugi_Unit_Cr

https://myanimelist.net/anime/1641/Iron_Virgin_Jun
https://myanimelist.net/anime/2972/Kindan_no_Mokushiroku__Crystal_Triangle
https://myanimelist.net/anime/11867/Cipher
https://myanimelist.net/anime/3930/Issunboushi_no_Shusse
https://myanimelist.net/anime/8925/Kobe_to_Watashi
https://myanimelist.net/anime/33693/Meitantei_Gordon
https://myanimelist.net/anime/5813/Urashima_Tarou_1931
https://myanimelist.net/anime/11357/Micro_Teukgongdae_Diatron_5
https://myanimelist.net/anime/3113/Pinky_Street
https://myanimelist.net/anime/7882/Ramen_Tenshi_Pretty_Menma
https://myanimelist.net/anime/31761/Sayokyoku
https://myanimelist.net/anime/18047/Seiyuu_Deka
https://myanimelist.net/anime/13421/Wheels
https://myanimelist.net/anime/28831/Anomalies
https://myanimelist.net/anime/22385/a_caFe
https://myanimelist.net/anime/7192/Choujikuu_Romanesque_Samy__Missing_99
https://myanimelist.net/anime/22415/Aoi_Heya
https://myanimelist.net/anime/35971/Onyankopon
https://myanimelist.net/anime/30339/Play_Jazz
https

https://myanimelist.net/anime/42260/Ninja_Collection
https://myanimelist.net/anime/7028/Pop
https://myanimelist.net/anime/31687/◯
https://myanimelist.net/anime/22157/Yasashii_Kinyoubi
https://myanimelist.net/anime/48680/Tesla_Note
https://myanimelist.net/anime/40282/Nendo_no_Tatakai_2019
https://myanimelist.net/anime/1585/Spectral_Force
https://myanimelist.net/anime/29984/Crayon_Angel
https://myanimelist.net/anime/3801/Yami_no_Teio__Kyuuketsuki_Dracula
https://myanimelist.net/anime/35448/Yo_Daitouryou_Trap-kun
https://myanimelist.net/anime/29619/Good-by_Marilyn
https://myanimelist.net/anime/6950/Aos
https://myanimelist.net/anime/28657/Red_Colored_Bridge
https://myanimelist.net/anime/7022/G-senjou_no_Higeki
https://myanimelist.net/anime/11689/CCW__Crazy_Clay_Wrestling
https://myanimelist.net/anime/16610/No_Littering
https://myanimelist.net/anime/22377/Wonder_Momo
https://myanimelist.net/anime/16602/Dead_Girl_Trailer
https://myanimelist.net/anime/34060/Dead_World
https://myanimelist.net/

KeyboardInterrupt: 

In [41]:
field_text

['Apr2010']

In [29]:
year

['2009',
 '2015',
 '2019',
 '2011',
 '2021',
 '2011',
 '2011',
 '1988',
 '2012',
 '20',
 '2017',
 '20',
 '2017',
 '20',
 '2008',
 '2006',
 '20',
 '2008',
 '2017',
 '2021',
 '20',
 '2018',
 '2021',
 '2021',
 '2018',
 '2016',
 '20',
 '2019',
 '2004',
 '20',
 '2013',
 '2020',
 '1998',
 '2021',
 '2017',
 '2014',
 '2021',
 '20',
 '2000',
 '2021',
 '2019',
 '1999']

In [30]:
a ='Jan 8, 2021'.split(',')[1]

In [31]:
a

' 2021'